# Aggregate Summary of Statistics and Make a Baseline Table

This notebook aggregates summary of statistics of all infection-related hospital admission data and processes them to make an aggregated baseline table.

In [9]:
import pandas as pd
import numpy as np

## 0- functions and read data

In [20]:
summary_table_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt.csv')
summary_table_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt.csv')
summary_table_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl.csv')
summary_table_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl.csv')
summary_table_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt.csv')
summary_table_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt.csv')
summary_table_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl.csv')
summary_table_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl.csv')
summary_table_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt.csv')
summary_table_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt.csv')
summary_table_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl.csv')
summary_table_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl.csv')
summary_table_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt.csv')
summary_table_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt.csv')
summary_table_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl.csv')
summary_table_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl.csv')
summary_table_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt.csv')
summary_table_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt.csv')
summary_table_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl.csv')
summary_table_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl.csv')
summary_table_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt.csv')
summary_table_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt.csv')
summary_table_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl.csv')
summary_table_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl.csv')
summary_table_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt.csv')
summary_table_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt.csv')
summary_table_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl.csv')
summary_table_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl.csv')
summary_table_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt.csv')
summary_table_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt.csv')
summary_table_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl.csv')
summary_table_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl.csv')
summary_table_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt.csv')
summary_table_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt.csv')
summary_table_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl.csv')
summary_table_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl.csv')
summary_table_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt.csv')
summary_table_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt.csv')
summary_table_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl.csv')
summary_table_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl.csv')

In [21]:
summary_table_lrti_no_abs_incdt

,variable,count,mean,std
0,antibacterial_brit,19665.0,1.226442,1.420304
1,age_cat_15_24,2305.0,0.143873,0.350972
2,age_cat_25_34,2335.0,0.145557,0.352672
3,age_cat_35_44,2205.0,0.137574,0.344463
4,age_cat_45_54,2335.0,0.145744,0.352860
5,age_cat_55_64,2250.0,0.140256,0.347263
6,age_cat_65_74,2315.0,0.144434,0.351540
7,age_cat_75_more,2285.0,0.142563,0.349638
8,sex_female,8095.0,0.504708,0.499993
9,sex_male,7940.0,0.495292,0.499993


In [26]:
print(summary_table_lrti_abs_incdt['variable'].tolist())

['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more', 'sex_female', 'sex_male', 'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low', 'flu_vaccine_no', 'flu_vaccine_yes', 'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown', 'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent', 'ethnicity_unknown', 'ethnicity_white', 'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown', 'season_autumn', 'season_spring', 'season_summer', 'season_winter', 'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic', 'antibacterial_brit', nan, 'Total

In [23]:
def data_process(data):
    # #replace ab_date with ab_binary in variable column
    # data.loc[data['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'
    #replace SMs with 5 in data
    for row in range(0, len(data)):
        if data['count'][row] == 'SM':
            data['count'][row] = 5
    #conver counts into int
    data['count_int'] = data['count'].astype(int)#.astype(str).replace('\.0', '', regex=True).astype(int)
    #convert means into percentages with two decimals
    data['mean_perc'] = data['mean']*100
    data['mean_perc'] = np.round(data['mean_perc'], decimals=2)
    #custom order of variables to fit tables of the manuscript
    data['variable'] = pd.Categorical(data['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more', 'sex_female', 'sex_male', 'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low', 'flu_vaccine_no', 'flu_vaccine_yes', 'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown', 'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent', 'ethnicity_white', 'ethnicity_asian', 'ethnicity_black', 'ethnicity_mixed', 'ethnicity_other', 'ethnicity_unknown', 'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown', 'season_autumn', 'season_spring', 'season_summer', 'season_winter', 'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic', 'antibacterial_brit'])
    data = data.sort_values('variable')
    #keep mean values of gp_count and antibacterial_brit
    data['mean'] = np.round(data['mean'], decimals=2)
    #reset index to prevent issue in calculating total value
    data = data.reset_index(drop=True)
    #calculate total from sex_male and sex_female
    total = data.iloc[data[data['variable']=='sex_female'].index[0],data.columns.get_loc('count')] + data.iloc[data[data['variable']=='sex_male'].index[0],data.columns.get_loc('count')]
    #create a new dataframe for the new row and add comma to the total's thousands
    new_row = pd.DataFrame({'variable':'Total', 'count':'-', 'mean':'-', 'std':'-', 'count_int':'-', 'mean_perc':'-', 'count_mean':total}, index=[0])
    new_row['count_mean'] = new_row['count_mean'].astype(int)
    new_row['count_mean'] = new_row['count_mean'].apply(lambda x : "{:,}".format(x))
    #keep 2 digits of mean_perc
    data['mean_perc'] = data['mean_perc'].apply(lambda x: "{:.2f}".format(x))
    #add comma after 3 digits
    data['count_int'] = data['count_int'].astype(int)
    data['count_int'] = data['count_int'].apply(lambda x : "{:,}".format(x))
    #replace SN
    for row in range(0, len(data)):
        if data['count'][row] == 5:
            data['count_int'][row] = 'SN'
            data['mean_perc'][row] = 'SN'
    #combine counts and means
    data['count_mean'] = data['count_int'].astype(str) + ' (' + data['mean_perc'].astype(str) + ')'
    #append row to the dataframe
    data = pd.concat([data, new_row])
    #replace
    data['std_round'] = pd.to_numeric(data['std'],errors='coerce')
    data['std_round'] = np.round(data['std_round'], decimals=2)
    data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('count_mean'))] = str(data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('mean'))]) + ' (' + str(data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('std_round'))]) + ')'

    return data

## 1- process data to make basline table, and save it

In [24]:
summary_table_lrti_no_abs_incdt = data_process(summary_table_lrti_no_abs_incdt)
summary_table_lrti_abs_incdt = data_process(summary_table_lrti_abs_incdt)
summary_table_lrti_no_abs_prevl = data_process(summary_table_lrti_no_abs_prevl)
summary_table_lrti_abs_prevl = data_process(summary_table_lrti_abs_prevl)
summary_table_urti_combined_no_abs_incdt = data_process(summary_table_urti_combined_no_abs_incdt)
summary_table_urti_combined_abs_incdt = data_process(summary_table_urti_combined_abs_incdt)
summary_table_urti_combined_no_abs_prevl = data_process(summary_table_urti_combined_no_abs_prevl)
summary_table_urti_combined_abs_prevl = data_process(summary_table_urti_combined_abs_prevl)
summary_table_uti_no_abs_incdt = data_process(summary_table_uti_no_abs_incdt)
summary_table_uti_abs_incdt = data_process(summary_table_uti_abs_incdt)
summary_table_uti_no_abs_prevl = data_process(summary_table_uti_no_abs_prevl)
summary_table_uti_abs_prevl = data_process(summary_table_uti_abs_prevl)
summary_table_sinusitis_no_abs_incdt = data_process(summary_table_sinusitis_no_abs_incdt)
summary_table_sinusitis_abs_incdt = data_process(summary_table_sinusitis_abs_incdt)
summary_table_sinusitis_no_abs_prevl = data_process(summary_table_sinusitis_no_abs_prevl)
summary_table_sinusitis_abs_prevl = data_process(summary_table_sinusitis_abs_prevl)
summary_table_ot_externa_no_abs_incdt = data_process(summary_table_ot_externa_no_abs_incdt)
summary_table_ot_externa_abs_incdt = data_process(summary_table_ot_externa_abs_incdt)
summary_table_ot_externa_no_abs_prevl = data_process(summary_table_ot_externa_no_abs_prevl)
summary_table_ot_externa_abs_prevl = data_process(summary_table_ot_externa_abs_prevl)
summary_table_otmedia_no_abs_incdt = data_process(summary_table_otmedia_no_abs_incdt)
summary_table_otmedia_abs_incdt = data_process(summary_table_otmedia_abs_incdt)
summary_table_otmedia_no_abs_prevl = data_process(summary_table_otmedia_no_abs_prevl)
summary_table_otmedia_abs_prevl = data_process(summary_table_otmedia_abs_prevl)
summary_table_urti_no_abs_incdt = data_process(summary_table_urti_no_abs_incdt)
summary_table_urti_abs_incdt = data_process(summary_table_urti_abs_incdt)
summary_table_urti_no_abs_prevl = data_process(summary_table_urti_no_abs_prevl)
summary_table_urti_abs_prevl = data_process(summary_table_urti_abs_prevl)
summary_table_cough_no_abs_incdt = data_process(summary_table_cough_no_abs_incdt)
summary_table_cough_abs_incdt = data_process(summary_table_cough_abs_incdt)
summary_table_cough_no_abs_prevl = data_process(summary_table_cough_no_abs_prevl)
summary_table_cough_abs_prevl = data_process(summary_table_cough_abs_prevl)
summary_table_cough_cold_no_abs_incdt = data_process(summary_table_cough_cold_no_abs_incdt)
summary_table_cough_cold_abs_incdt = data_process(summary_table_cough_cold_abs_incdt)
summary_table_cough_cold_no_abs_prevl = data_process(summary_table_cough_cold_no_abs_prevl)
summary_table_cough_cold_abs_prevl = data_process(summary_table_cough_cold_abs_prevl)
summary_table_throat_no_abs_incdt = data_process(summary_table_throat_no_abs_incdt)
summary_table_throat_abs_incdt = data_process(summary_table_throat_abs_incdt)
summary_table_throat_no_abs_prevl = data_process(summary_table_throat_no_abs_prevl)
summary_table_throat_abs_prevl = data_process(summary_table_throat_abs_prevl)

In [25]:
summary_table_lrti_no_abs_incdt

,variable,count,mean,std,count_int,mean_perc,count_mean,std_round
0,age_cat_15_24,20615.0,0.14,0.349727,"20,615",14.27,"20,615 (14.27)",0.35
1,age_cat_25_34,20475.0,0.14,0.348735,"20,475",14.17,"20,475 (14.17)",0.35
2,age_cat_35_44,20575.0,0.14,0.349437,"20,575",14.24,"20,575 (14.24)",0.35
3,age_cat_45_54,20610.0,0.14,0.349685,"20,610",14.26,"20,610 (14.26)",0.35
4,age_cat_55_64,20780.0,0.14,0.35089,"20,780",14.38,"20,780 (14.38)",0.35
5,age_cat_65_74,20575.0,0.14,0.349423,"20,575",14.24,"20,575 (14.24)",0.35
6,age_cat_75_more,20880.0,0.14,0.351584,"20,880",14.45,"20,880 (14.45)",0.35
7,sex_female,73710.0,0.51,0.499901,"73,710",51.00,"73,710 (51.00)",0.50
8,sex_male,70810.0,0.49,0.499901,"70,810",49.00,"70,810 (49.00)",0.50
9,CCI_cat_high,15155.0,0.1,0.306399,"15,155",10.49,"15,155 (10.49)",0.31


In [18]:
d = {'variable': summary_table_lrti_no_abs_incdt['variable'],
     
     'lrti_no_abs_incdt': summary_table_lrti_no_abs_incdt['count_mean'],
     'lrti_abs_incdt': summary_table_lrti_abs_incdt['count_mean'],
     'lrti_no_abs_prevl': summary_table_lrti_no_abs_prevl['count_mean'],
     'lrti_abs_prevl': summary_table_lrti_abs_prevl['count_mean'],

     'urti_combined_no_abs_incdt': summary_table_urti_combined_no_abs_incdt['count_mean'],
     'urti_combined_abs_incdt': summary_table_urti_combined_abs_incdt['count_mean'],
     'urti_combined_no_abs_prevl': summary_table_urti_combined_no_abs_prevl['count_mean'],
     'urti_combined_abs_prevl': summary_table_urti_combined_abs_prevl['count_mean'],

    'uti_no_abs_incdt': summary_table_uti_no_abs_incdt['count_mean'],
    'uti_abs_incdt': summary_table_uti_abs_incdt['count_mean'],
    'uti_no_abs_prevl': summary_table_uti_no_abs_prevl['count_mean'],
    'uti_abs_prevl': summary_table_uti_abs_prevl['count_mean'],

    'sinusitis_no_abs_incdt': summary_table_sinusitis_no_abs_incdt['count_mean'],
    'sinusitis_abs_incdt': summary_table_sinusitis_abs_incdt['count_mean'],
    'sinusitis_no_abs_prevl': summary_table_sinusitis_no_abs_prevl['count_mean'],
    'sinusitis_abs_prevl': summary_table_sinusitis_abs_prevl['count_mean'],

    'ot_externa_no_abs_incdt': summary_table_ot_externa_no_abs_incdt['count_mean'],
    'ot_externa_abs_incdt': summary_table_ot_externa_abs_incdt['count_mean'],
    'ot_externa_no_abs_prevl': summary_table_ot_externa_no_abs_prevl['count_mean'],
    'ot_externa_abs_prevl': summary_table_ot_externa_abs_prevl['count_mean'],

    'otmedia_no_abs_incdt': summary_table_otmedia_no_abs_incdt['count_mean'],
    'otmedia_abs_incdt': summary_table_otmedia_abs_incdt['count_mean'],
    'otmedia_no_abs_prevl': summary_table_otmedia_no_abs_prevl['count_mean'],
    'otmedia_abs_prevl': summary_table_otmedia_abs_prevl['count_mean'],

    'urti_no_abs_incdt': summary_table_urti_no_abs_incdt['count_mean'],
    'urti_abs_incdt': summary_table_urti_abs_incdt['count_mean'],
    'urti_no_abs_prevl': summary_table_urti_no_abs_prevl['count_mean'],
    'urti_abs_prevl': summary_table_urti_abs_prevl['count_mean'],

    'cough_no_abs_incdt': summary_table_cough_no_abs_incdt['count_mean'],
    'cough_abs_incdt': summary_table_cough_abs_incdt['count_mean'],
    'cough_no_abs_prevl': summary_table_cough_no_abs_prevl['count_mean'],
    'cough_abs_prevl': summary_table_cough_abs_prevl['count_mean'],

    'cough_cold_no_abs_incdt': summary_table_cough_cold_no_abs_incdt['count_mean'],
    'cough_cold_abs_incdt': summary_table_cough_cold_abs_incdt['count_mean'],
    'cough_cold_no_abs_prevl': summary_table_cough_cold_no_abs_prevl['count_mean'],
    'cough_cold_abs_prevl': summary_table_cough_cold_abs_prevl['count_mean'],

    'throat_no_abs_incdt': summary_table_throat_no_abs_incdt['count_mean'],
    'throat_abs_incdt': summary_table_throat_abs_incdt['count_mean'],
    'throat_no_abs_prevl': summary_table_throat_no_abs_prevl['count_mean'],
    'throat_abs_prevl': summary_table_throat_abs_prevl['count_mean'],
     
    }
data = pd.DataFrame(data=d)

data.to_csv(f'../output/aggregate_tables/baseline_table_all_infections.csv', index=False)
data

,variable,lrti_no_abs_incdt,lrti_abs_incdt,lrti_no_abs_prevl,lrti_abs_prevl
0,age_cat_15_24,"20,615 (14.27)","2,305 (14.39)","2,345 (14.44)",245 (13.75)
1,age_cat_25_34,"20,475 (14.17)","2,335 (14.56)","2,300 (14.17)",250 (13.92)
2,age_cat_35_44,"20,575 (14.24)","2,205 (13.76)","2,335 (14.37)",265 (14.70)
3,age_cat_45_54,"20,610 (14.26)","2,335 (14.57)","2,305 (14.20)",270 (15.15)
4,age_cat_55_64,"20,780 (14.38)","2,250 (14.03)","2,245 (13.83)",260 (14.48)
5,age_cat_65_74,"20,575 (14.24)","2,315 (14.44)","2,310 (14.24)",245 (13.64)
6,age_cat_75_more,"20,880 (14.45)","2,285 (14.26)","2,395 (14.76)",255 (14.37)
7,sex_female,"73,710 (51.00)","8,095 (50.47)","8,255 (50.85)",870 (48.69)
8,sex_male,"70,810 (49.00)","7,940 (49.53)","7,980 (49.15)",920 (51.31)
9,CCI_cat_high,"15,155 (10.49)","1,745 (10.88)","1,680 (10.35)",175 (9.78)


In [ ]:
data[['urti_combined_no_abs_incdt', 'urti_combined_abs_incdt', 'urti_combined_no_abs_prevl', 'urti_combined_abs_prevl']].loc[data['variable'] == 'Total']

In [ ]:
data[['urti_no_abs_incdt', 'urti_abs_incdt', 'urti_no_abs_prevl', 'urti_abs_prevl']].loc[data['variable'] == 'Total']

In [ ]:
data[['cough_no_abs_incdt', 'cough_abs_incdt', 'cough_no_abs_prevl', 'cough_abs_prevl']].loc[data['variable'] == 'Total']

In [ ]:
data[['cough_cold_no_abs_incdt', 'cough_cold_abs_incdt', 'cough_cold_no_abs_prevl', 'cough_cold_abs_prevl']].loc[data['variable'] == 'Total']

In [ ]:
data[['throat_no_abs_incdt', 'throat_abs_incdt', 'throat_no_abs_prevl', 'throat_abs_prevl']].loc[data['variable'] == 'Total']